In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import regularizers

In [11]:
import warnings

# Suppress all warnings
warnings.filterwarnings("ignore")

In [12]:

# Load the CSV file into a DataFrame
df = pd.read_csv('pre_processed_data.csv')

In [13]:

# Preprocess the data
df = df[df['admit'] != 0]
X = df.drop(columns=['userName', 'major', "univName", "admit", 'greV', 'greQ'])
y = df["univName"]
print("Number of unique universities:", df['univName'].nunique())

# Split the dataset into features (X) and target labels (y)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Impute missing values with the mean
X_train.fillna(X_train.mean(), inplace=True)

# Replace infinite values with NaN
X_train.replace([np.inf, -np.inf], np.nan, inplace=True)

# Impute missing values with the mean
X_train.fillna(X_train.mean(), inplace=True)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Number of unique universities: 54


In [14]:

# Encode categorical labels
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Convert labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)
y_test_one_hot = to_categorical(y_test_encoded, num_classes=num_classes)

# Build the ANN model
model = Sequential([
    Dense(256, input_shape=(X_train_scaled.shape[1],), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    Dropout(0.5),  # Dropout with 50% probability
    Dense(128, activation='relu'),
    Dropout(0.4),  # Dropout with 40% probability
    Dense(64, activation='relu'),
    Dropout(0.3),  # Dropout with 30% probability
    Dense(32, activation='relu'),
    Dropout(0.3),  # Dropout with 30% probability
    Dense(16, activation='relu'),
    Dropout(0.2),  # Dropout with 20% probability
    Dense(8, activation='relu'),
    Dropout(0.2),  # Dropout with 20% probability
    Dense(num_classes, activation='softmax')
])
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_scaled, y_train_one_hot, epochs=500, batch_size=32, validation_split=0.2)

# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(X_test_scaled, y_test_one_hot)
print(f'Test Accuracy: {test_accuracy}')

# Save the trained model
model.save('ann_model.h5')


Epoch 1/500
517/517 [==============================] - 1s 2ms/step - loss: 3.4596 - accuracy: 0.1093 - val_loss: 2.8486 - val_accuracy: 0.2224
Epoch 2/500
517/517 [==============================] - 1s 2ms/step - loss: 2.8286 - accuracy: 0.2080 - val_loss: 2.4511 - val_accuracy: 0.2659
Epoch 3/500
517/517 [==============================] - 1s 2ms/step - loss: 2.5999 - accuracy: 0.2442 - val_loss: 2.2640 - val_accuracy: 0.3213
Epoch 4/500
517/517 [==============================] - 1s 2ms/step - loss: 2.4594 - accuracy: 0.2636 - val_loss: 2.1371 - val_accuracy: 0.3646
Epoch 5/500
517/517 [==============================] - 1s 2ms/step - loss: 2.3709 - accuracy: 0.2778 - val_loss: 2.0758 - val_accuracy: 0.3254
Epoch 6/500
517/517 [==============================] - 1s 2ms/step - loss: 2.2836 - accuracy: 0.2975 - val_loss: 1.9277 - val_accuracy: 0.3917
Epoch 7/500
517/517 [==============================] - 1s 1ms/step - loss: 2.1769 - accuracy: 0.3295 - val_loss: 1.7948 - val_accuracy: 0.4147